In [1]:
import pandasql as ps
import vk
import vk_api
from datetime import datetime
import csv
import pandas as pd

In [2]:
# Ваш сервисный токен и ID приложения
token = ''
app_id = ''

In [3]:
# Айди таргетированного пользователя
# Шестаков Данил = 224459838
# Александр Ласточкин = 554042102
user_id = 224459838

In [4]:
# Инициализируем VK API с помощью сервисного токена
vk_session = vk_api.VkApi(token=token, app_id=app_id)

# Получаем объект VK API
vk = vk_session.get_api()

# проверяем работу сессии
vk.users.get(user_ids=user_id)

[{'id': 224459838,
  'first_name': 'Danil',
  'last_name': 'Shestakov',
  'can_access_closed': True,
  'is_closed': False}]

In [5]:
# Получаем список всех постов пользователя
response = vk_session.method('wall.get', {'owner_id': user_id, 'count': 100})

# Обрабатываем ответ от VK API
if 'items' in response:
    posts = response['items']
else:
    print('Error occurred while getting posts:', response)

In [6]:
# Проверяем внутренности post'а
posts[0]

{'is_pinned': 1,
 'comments': {'can_post': 1, 'count': 0, 'groups_can_post': True},
 'hash': 'p0DUMzlJgOq0uDv0ww',
 'type': 'post',
 'attachments': [{'type': 'video',
   'video': {'response_type': 'full',
    'access_key': '26f302e29c5c5c8219',
    'can_like': 1,
    'can_repost': 1,
    'can_subscribe': 1,
    'can_add_to_faves': 0,
    'can_add': 1,
    'comments': 0,
    'date': 1664139981,
    'description': 'https://www.youtube.com/c/PlanyaCh - Автор кавера / Author this cover\n#weplanet #anime #war #history #planya #аниме #планя #worldwar #song #katyusha \n#песня \nАвтор Слов - Михаил Исаковский \nАвтор Музыки - Матвей Блантер \nПравообладатель - Грибоедова Наталья Никандровна  \nhttps://rao.ru/information/reestry/reestr-proizvedenij-rossijskih-pravoobladatelej/?work&author=%D0%93%D1%80%D0%B8%D0%B1%D0%BE%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%20%D0%9D%D0%B0%D1%82%D0%B0%D0%BB%D1%8C%D1%8F%20%D0%9D%D0%B8%D0%BA%D0%B0%D0%BD%D0%B4%D1%80%D0%BE%D0%B2%D0%BD%D0%B0 \n\nВсе материалы, использованные 

In [7]:
posts[0]["likes"]["count"] # выбираем количество лайков с поста

3

In [8]:
posts[0]["id"] # выбираем айдишник поста

24555

In [9]:
posts[0]["date"] # выбираем дату поста из джейсона 

1664139981

In [10]:
datetime.fromtimestamp(posts[0]["date"]).strftime("%A, %B, %d, %Y, %H:%M:%S").split(", ")

['Monday', 'September', '26', '2022', '00:06:21']

In [11]:
with open('posts.csv', 'w', newline='') as csvfile:
    fieldnames = ['post_id', 'likes_count', 'post_datetime']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for post in posts:
        writer.writerow({'post_id': post["id"], 'likes_count': post["likes"]["count"], 'post_datetime': post["date"]})

In [12]:
datafr = pd.read_csv("posts.csv")

In [13]:
datafr.head(5)

,post_id,likes_count,post_datetime
0,24555,3,1664139981
1,24869,0,1690655159
2,24868,1,1690573794
3,24867,1,1690551112
4,24865,0,1690227353


In [14]:
def time_of_day_to_word(hour:str):
    try:
        hour = int(hour)
    except:
        return 'day'
    if 0 <= hour < 6:
        return 'night'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour <18:
        return 'day'
    elif 18 <= hour <= 23:
        return 'evening'
    else:
        return 'day'

In [90]:
# если версия питона, начиная с 3.10
def time_of_day_to_word_match (hour):
    #hour = int(hour)
    match hour:
        case 0 <= hour < 6:
            return 'night'
        case 6 < hour < 12:
            return 'morning'
        case 12 <= hour <18:
            return 'day'
        case 18 <= hour <= 23:
            return 'evening'
        case _:
            default('day')

SyntaxError: invalid syntax (2365479714.py, line 4)

In [15]:
datafr["week_day"] = datafr['post_datetime'].apply(lambda x: datetime.fromtimestamp(x).strftime("%A"))

In [16]:
datafr["time_of_day"] = datafr['post_datetime'].apply(lambda x: datetime.fromtimestamp(x).strftime("%H"))

datafr["time_of_day"] = datafr["time_of_day"].apply(time_of_day_to_word)

In [17]:
datafr

,post_id,likes_count,post_datetime,week_day,time_of_day
0,24555,3,1664139981,Monday,night
1,24869,0,1690655159,Saturday,evening
2,24868,1,1690573794,Friday,evening
3,24867,1,1690551112,Friday,day
4,24865,0,1690227353,Monday,evening
...,...,...,...,...,...
95,24773,0,1678398307,Friday,night
96,24772,0,1678313308,Thursday,night
97,24771,0,1678301038,Wednesday,evening
98,24770,0,1678213921,Tuesday,evening


In [42]:
# среднее количество лайков для пар записей, у которых промежуток времени между постами больше нуля
# таким образом, мы находим корреляцию между временной дистанцией между постами и количеством лайков 
sql_query = '''
SELECT AVG(likes_count) 
FROM (
  SELECT t1.post_id, t1.likes_count, MIN(t2.post_datetime) - t1.post_datetime AS time_diff
    FROM datafr as t1
    INNER JOIN datafr as t2 ON t1.post_id < t2.post_id
    GROUP BY t1.post_id, t1.likes_count
  ) subquery
WHERE time_diff > 0
    '''

In [43]:
ps.sqldf(sql_query)

,AVG(likes_count)
0,0.565657


In [117]:
# среднее количество лайков для каждого дня недели
sql_query = '''
SELECT week_day, AVG(likes_count) AS avg_likes
FROM datafr
GROUP BY week_day
    '''

In [118]:
ps.sqldf(sql_query)

,week_day,avg_likes
0,Friday,0.535714
1,Monday,1.066667
2,Saturday,0.250000
3,Sunday,0.500000
4,Thursday,0.461538
5,Tuesday,0.500000
6,Wednesday,0.416667


In [119]:
# подсчитает среднее количество лайков для каждого значения поля time_of_day т.е. дня недели
sql_query = '''
SELECT time_of_day, AVG(likes_count) AS avg_likes
FROM datafr
GROUP BY time_of_day
    '''

In [120]:
ps.sqldf(sql_query)

,time_of_day,avg_likes
0,day,0.476190
1,evening,0.540541
2,morning,1.142857
3,night,0.571429
